# Step #1: Parse through JSON Flight Log File

In [22]:
import json
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np

# Enter Flight Log JSON from Fly Catcher BELOW
#selected_file = "./samples/spoofed/aircraft_spoofed.json" 
#selected_file = "./samples/valid/aircraft_valid_small.json" 
selected_file = "./samples/testing/samplecopy.json"

file = open(selected_file)
json_contents = json.load(file)
aircraft = json_contents["aircraft"]

planes = []

class PlaneLog:
    def __init__(self, hex):
        self.hex = hex
        self.planes = []
    def add_plane(self, plane_record):
        self.planes.append(plane_record)

for record in aircraft:
    matched_plane = False
    for plane in planes:
        if plane.hex == record["hex"]:
            plane.add_plane(record)
            matched_plane = True
    if matched_plane == False:
        new_plane = PlaneLog(record["hex"])
        new_plane.add_plane(record)
        planes.append(new_plane)
            
file.close()

print(f"🛫 Detected {len(planes)} planes!")
for i, plane in enumerate(planes):
    print(f"Plane #{i + 1}: {plane.hex} - {len(plane.planes)} records")

def find_plane(hex):
    for plane in planes:
        if plane.hex == hex:
            return plane

🛫 Detected 20 planes!
Plane #1: 845f9f - 1 records
Plane #2: 86d624 - 1 records
Plane #3: a835c5 - 1 records
Plane #4: 4d0104 - 1 records
Plane #5: 86e4da - 1 records
Plane #6: 71c083 - 1 records
Plane #7: 76cdb9 - 1 records
Plane #8: 7c1479 - 1 records
Plane #9: 86ebb6 - 1 records
Plane #10: abba6c - 1 records
Plane #11: 7c531d - 1 records
Plane #12: ab271f - 1 records
Plane #13: 8694fa - 1 records
Plane #14: ADCF46 - 1 records
Plane #15: B49A8B - 1 records
Plane #16: B96D38 - 1 records
Plane #17: E63452 - 1 records
Plane #18: B9FE57 - 1 records
Plane #19: 8F79E5 - 1 records
Plane #20: B0BB00 - 1 records


# Step 2: Load the CNN Model

In [23]:
model = tf.keras.models.load_model("Spoof_DetectionV3.keras")
model.summary()
print(model.input_shape, model.output_shape)
def handle_alt_baro(alt_baro):
    if alt_baro == "ground":
        return 0.0
    else:
        return alt_baro

def predict_adsb_data(adsb_message):
    try:
        feature_vector = [
            handle_alt_baro(adsb_message.get('alt_baro', 0)),
            #float(adsb_message.get('alt_baro', 0)),
            float(adsb_message.get('gs', 0)),
            float(adsb_message.get('track', 0)),
            float(adsb_message.get('baro_rate', 0)),
            float(adsb_message.get('lat', 0)),
            float(adsb_message.get('lon', 0)),
            float(adsb_message.get('seen_pos', 0)),
            float(adsb_message.get('messages', 0)),
            float(adsb_message.get('seen', 0)),
            float(adsb_message.get('rssi', 0)),
        ]
    except ValueError as e:
        print(f"Error processing ADS-B message: {adsb_message}, Field: {e}")
        return None

    # scaler = StandardScaler()
    # features_scaled = scaler.fit_transform([feature_vector])
    # prediction = model.predict(np.array(features_scaled))

    prediction = model.predict(np.array([feature_vector]))
    return prediction[0][0]
    #return int(round(prediction[0][0]))

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_44 (Dense)                │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,221 (36.02 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 6,020 (23.52 KB)

(None, 10) (None, 1)


# Step 3: Check if aircraft is spoofed and display on map.

In [24]:
import folium
from IPython.display import display
m = folium.Map(location = [51.5800, 5.1875], tiles ='OpenStreetMap', zoom_start=3, max_bounds=True, min_zoom=2)

#scaler = StandardScaler()

for plane in planes:
    print(plane.hex)
    aircraft_data = find_plane(plane.hex)
    if aircraft_data == None:
        continue
        
    results = predict_adsb_data(aircraft_data.planes[0])
    print(f'[RESULTS: {results}]')
    spoofed = (True if results > 0.5 else False)
    percentage = results * 100

    text = ("Spoofed" if spoofed else "Not Spoofed")
    print(f"{text} -#-- {percentage}% prediction of being spoofed")
    
    plane_values = [{"key": "hex", "value": "Hex (24-bit ICAO identifier)"}, {"key": "type", "value": "Data Source"}, {"key": "flight", "value": "Flight Callsign"}, {"key": "r", "value": "Aircraft Registration"}, {"key": "t", "value": "Aircraft Type"}, {"key": "alt_baro", "value": "Aircraft Barometric Altitude (feet)"}, {"key": "gs", "value": "Ground Speed (knots)"}, {"key": "track", "value": "True Track (degrees)"}, {"key": "baro_rate", "value": "Rate of Change of Barometric Altitude (feet/minute)"}, {"key": "lat", "value": "Latitude"}, {"key": "lon", "value": "Longitude"}, {"key": "seen_pos", "value": "Updated Time Ago (seconds)"}, {"key": "messages", "value": "Total # of Mode S Messages"}, {"key": "seen", "value": "Last Timing Message (seconds)"}, {"key": "rssi", "value": "Signal Power (dbFs)"}]
    
    def find_value(id):
        for x in plane_values:
            if x["key"] == id:
                return x

    plane_data = aircraft_data.planes[0]
    plane_coordinates = (plane_data["lat"], plane_data["lon"])

    # Draw map markers
    folium.CircleMarker(
        location=plane_coordinates,
        radius=5,
        color=("red" if spoofed else "green"),
        fill=True,
        fill_color=("red" if spoofed else "green"),
        fill_opacity=0.7,
        popup='Flight: ' + aircraft_data.planes[0]['flight'] + '<br>Hex: ' + plane.hex + '<br>Lat: ' + str(aircraft_data.planes[0]['lat']) + '<br>Lon: ' + str(aircraft_data.planes[0]['lon'])
    ).add_to(m)

display(m)

845f9f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[RESULTS: -228995.109375]
Not Spoofed -#-- -22899510.0% prediction of being spoofed
86d624
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[RESULTS: -140789.390625]
Not Spoofed -#-- -14078939.0% prediction of being spoofed
a835c5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[RESULTS: -256560.71875]
Not Spoofed -#-- -25656072.0% prediction of being spoofed
4d0104
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[RESULTS: -356171.15625]
Not Spoofed -#-- -35617116.0% prediction of being spoofed
86e4da
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[RESULTS: -175618.703125]
Not Spoofed -#-- -17561870.0% prediction of being spoofed
71c083
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[RESULTS: -221635.046875]
Not Spoofed -#-- -22163504.0% prediction of being spoofed
76cdb9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[RESULTS: -195319.921875]
Not Spoofed -#-- -19531992.0% prediction of being spoofed
7c1479
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[RESULTS: -145454.34375]
Not Spoofed -#-- -14545434.0%